In [76]:
# 폰트 설정 
import matplotlib.pyplot as plt
import matplotlib as mpl

# WINDOWS
plt.rcParams['font.family'] = 'Malgun Gothic'
mpl.rcParams['axes.unicode_minus'] = False

In [77]:
from dotenv import load_dotenv

load_dotenv()

True

In [78]:
from langchain_teddynote import logging

logging.langsmith("Marketability_Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Marketability_Agent


# 시장성 분석 에이전트
: 시장 성장성, 수요 분석
- 시장 리포트, 산업 뉴스 검색 등
- 기업, 도메인 

In [79]:
from typing import Dict, List, TypedDict, Annotated, Sequence, Literal
from langgraph.graph.message import add_messages # 기존 메시지에 메시지를 더한다.

from langchain_core.messages import HumanMessage, BaseMessage,AIMessage, SystemMessage, ToolMessage

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent, tools_condition
from langchain.tools import tool
import operator

In [80]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_teddynote.models import get_model_name, LLMs

import uuid

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END

# 상태 정의 & 에이전트 정의

In [81]:
class AgentState(TypedDict):
    domain: Annotated[str, "Domain"]
    country: Annotated[str, "Country"]

    startup_list: Annotated[list[str], "Startup_list"]   # 스타트업 탐색 에이전트가 생성하는 주요 기업명 목록
    startup_profiles: Annotated[dict[str, dict], "Startup_profiles"]   # 스타트업별 정보 종합 저장소
    tech_summary: Annotated[dict[str, str], "Tech_summary"]  # 각 스타트업 기술 요약 정보
    founder_reputation: Annotated[dict[str, dict], "Founder_reputation"]  # 창업자 이력 + 평판 정보
    market_analysis: Annotated[dict[str, dict], "Market_analysis"]  # 시장성 분석 결과
    legal_risk: Annotated[dict[str, str], "Legal_risk"]  # 법적/규제 이슈 요약
    competitor_info: Annotated[dict[str, dict], "Competitor_info"]  # 경쟁사 비교 분석
    investment_decision: Annotated[dict[str, str], "Investment_decision"]  # 투자 판단 (투자 / 보류 + 사유)
    final_report: Annotated[str, "Final_report"]  # 보고서 생성 에이전트의 출력물 (PDF or Text)

## 시장 성장성을 위한 산업 뉴스 검색 tool : tavily, naver news

In [ ]:
tavily_tool=TavilySearchResults(max_results=5)

@tool
def market_research(query: str)-> str:
     """
    industry news, 시장 성장성을 검색해 조사해옵니다.
    
    Args:
        query: 검색할 도메인이나 키워드
    """
     results = tavily_tool.invoke(f"{query} market trends analysis")
     return str(results)

## 수요 조사를 위한 검색 tool 

In [83]:
@tool
def look_for_demand(query: str)-> str:
    """
    시장에 대한 수요를 분석합니다.

    Args:
        query: 검색할 도메인이나 키워드 
    """
    results = tavily_tool.invoke(f"{query} market demand analysis")
    return f"{query}에 대한 시장 수요 분석 결과:\n{results}"

## 시장성 분석 에이전트

In [ ]:
def market_analysis(state: AgentState):
    """
    일반적인 시장 분석을 수행
    """

    domain= state["domain"]
    country= state["country"]
    print(domain)

    market_tools = [tavily_tool, look_for_demand, market_research]
    market_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    
    market_analysis_system_prompt = f"""
    당신은 스타트업을 위해 {domain}에 대한 종합적인 시장 분석을 전문으로 하는 전문 시장 분석가입니다.
    당신의 임무는 스타트업의 전망을 알아보기 위해 시장 동향, 시장 수요를 분석하는 것 입니다.

    다음 단계를 따르세요.
    1. market_research 사용해 {domain} 도메인의 시장 동향 및 업계 뉴스에 대한 정보 수집
    2. market_research를 통해 충분하지 못한 정보를 가져왔다고 판단되면 tavily_tool로 추가적인 정보를 수집
    3. tavily_tool 사용해 해당 {domain} 도메인에 대한 시장 수요에 대한 정보 수집
    4. 종합적인 시장 분석 정리. 수집한 정보를 28문장에서 32문장으로 요약하세요.
    5. 시장 기회를 평가하기 위해 시장 크기, 성장 가능성, 고객의 반응 예측, 수요 파악의 내용을 종합적인 시장 분석 정리에 추가하세요.
        모든 결론에는 수집한 정보에서 찾은 근거가 있어야 합니다.
    6. 한국어로 작성해 주세요.

    당신의 분석은 스타트업의 미래를 위한 시장 분석에 기여합니다.
    """
    
    market_agent = create_react_agent(
        market_llm,
        tools=market_tools,
        state_modifier=market_analysis_system_prompt,
    )
    
    # 초기화되지 않은 경우 "시장성 평가 결과 저장" 딕셔너리 초기화
    if "market_analysis" not in state or state["market_analysis"] is None:
        state["market_analysis"] = {}
    
    # 시장 분석 실행
    result = market_agent.invoke({
        "input": f"{domain} 도메인/산업의 시장 분석을 수행해주세요. 특히 {country} 시장에 초점을 맞춰주세요."
    })
    

    print("==========================================\n\n")
    print(result)
    print("==========================================\n\n")

    # 결과 처리 - AIMessage 형태의 최종 응답 확인
    final_responses = [msg for msg in result.get("messages", []) 
                      if isinstance(msg, AIMessage) and not msg.additional_kwargs.get("tool_calls")]
    
    if final_responses:
        analysis_output = final_responses[-1].content
    else:
        analysis_output = "시장 분석 결과를 얻지 못했습니다."
    
    # 분석 결과 구조화 -> 좋아보이니까 시간되면 하기
    # general_market_data = {
    #     "market_size": extract_market_info(analysis_output, "시장 크기"),
    #     "growth_potential": extract_market_info(analysis_output, "성장 가능성"),
    #     "customer_response": extract_market_info(analysis_output, "고객 반응"),
    #     "demand_analysis": extract_market_info(analysis_output, "수요 파악"),
    #     "market_analysis": analysis_output
    # }
    
    # 일반 시장 분석 결과 저장
    state["market_analysis"]["general"] = analysis_output
    
    return state


# 보고서 생성

In [85]:
def report(state: AgentState):
    """
    시장 분석 결과를 종합하여 최종 보고서를 생성하는 노드
    """
    report_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    
    report_prompt = f"""
    당신은 전문적인 시장 분석 보고서 작성가입니다. 
    다음 시장 분석 결과를 바탕으로 종합적인 보고서를 작성해 주세요.
    
    시장 정보: {state["domain"]}
    
    일반 시장 분석:
    {state["market_analysis"].get("general", {}).get("full_analysis", "정보 없음")}
    
    스타트업별 분석:
    """
    
    # 스타트업별 분석 정보 추가
    for startup in state["startup_list"]:
        if startup in state["market_analysis"]:
            report_prompt += f"\n{startup}:\n"
            report_prompt += f"{state['market_analysis'][startup].get('full_analysis', '정보 없음')}\n"
    
    report_prompt += """
    보고서에는 다음 내용을 포함해 주세요:
    1. 전체 시장 개요
    2. 주요 트렌드 및 성장 촉진 요인
    3. 각 스타트업의 시장 적합성 및 성장 가능성 평가
    4. 투자 관점에서의 시사점
    5. 결론 및 추천사항
    
    보고서는 전문적이고 객관적인 톤으로 작성해 주세요.
    """
    
    # 보고서 생성
    report_chain = report_llm | (lambda x: x.content)
    final_report = report_chain.invoke(report_prompt)
    
    # 최종 보고서 저장
    state["final_report"] = final_report
    
    # 사용자에게 보여줄 메시지 추가
    state["messages"] = add_messages(state["messages"], [
        AIMessage(content=f"# 최종 시장 분석 보고서\n\n{final_report}")
    ])
    
    return state

# Node & Edge 정의

In [86]:
memory=MemorySaver()
workflow=StateGraph(AgentState)

workflow.add_node("Market_Analyze", market_analysis)

workflow.add_edge("Market_Analyze", END)
workflow.add_edge(START, "Market_Analyze")

# Set the entry point
# workflow.set_entry_point("agent") # 스타트업 검색

graph=workflow.compile(checkpointer=memory)

In [87]:
# from langchain_teddynote.graphs import visualize_graph

# visualize_graph(graph)

In [88]:
config={"configurable": {"thread_id": str(uuid.uuid4())}}

In [89]:
def run_market_analysis(query: str):
    """
    Run the market analysis workflow with the given query.
    """
    # Initialize the state
    state = {
        "domain": query,
        "country": "미국",
        "startup_list": {},
        "startup_profiles": {},
        "tech_summary": {},
        "founder_reputation": {},
        "market_analysis": {},
        "legal_risk": {},
        "competitor_info": {},
        "investment_decision": {},
        "final_report": "",
    }

    result = graph.invoke(state, config)
    return result 

In [90]:
# Example usage
result = run_market_analysis(input())
print(result)

스포츠


{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_d3CmKBNVlkfsXfJxO6pTc28L', 'function': {'arguments': '{"query":"스포츠 시장 동향 및 업계 뉴스"}', 'name': 'market_research'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 392, 'total_tokens': 416, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BPK8V1atj2xWR9L6a3XuAek293NdN', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-353c938d-ec07-4655-afd8-5dc5900a1c0b-0', tool_calls=[{'name': 'market_research', 'args': {'query': '스포츠 시장 동향 및 업계 뉴스'}, 'id': 'call_d3CmKBNVlkfsXfJxO6pTc28L', 'type': 'tool_call'}], usage_metadata={'input_tokens': 392, 'output_tokens': 24, 'total_tokens': 416, '